###### Credits to hugging face documentation

In [1]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR) # This blocks all warnings

# Pipeline

The pipeline() is the easiest and fastest way to use a pretrained model for inference.
Start by creating an instance of pipeline()
The pipeline() downloads and caches a default pretrained model and tokenizer for sentiment analysis.
The pipeline() can accommodate any model from the Hub, making it easy to adapt the pipeline() for other use-cases.  

In [2]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis","distilbert-base-uncased-finetuned-sst-2-english")
classifier(["I don't think anybody hates cows.","I don't think anybody hate cows."]) # huh!

[{'label': 'NEGATIVE', 'score': 0.9911612272262573},
 {'label': 'POSITIVE', 'score': 0.9829936027526855}]

In [3]:
model = pipeline('text-generation', "gpt2")
print(type(model))
print(model("Last night, I saw a cow")[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<class 'transformers.pipelines.text_generation.TextGenerationPipeline'>


/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'generated_text': 'Last night, I saw a cow near the front gate of his residence which is a very old building. In the very same place, one evening my brother, who lived about ten or twelve years ago, came to look for a cow while at his'}


# AutomodelForCausalLM and AutoTokenizer

While pipeline() is an awesome way to use pre-trained models, it encapsulates all the working, like tokenising of the input, and back. To get all fine-grained control of the whole process, we use:

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Change the model_name to GPT-2
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Example text
text = "I saw a cow in the office, which"

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt")

# Generate text using GPT-2
generated_ids = model.generate(input_ids=inputs["input_ids"], max_length=50, num_return_sequences=1)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print the generated text
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: I saw a cow in the office, which was a little bit of a mess. I was like, 'What the hell is going on?' And he said, 'I'm going to take a look at it.' And I said, 'What


# Using GPU

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# initialising model and tokenizer for gpt2
gpt_model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)
gpt_tokenizer = AutoTokenizer.from_pretrained('gpt2')

input_data = 'There was a time when I saw a cow in the office'

# Encoding the input and moving tensors to GPU
encoding = gpt_tokenizer(input_data, return_tensors="pt").to(device)

# Generate text using GPT-2
generated_ids = gpt_model.generate(input_ids=encoding['input_ids'], max_length=50, num_return_sequences=1)

# Move generated IDs to CPU before decoding
generated_ids = generated_ids[0].cpu()

# Decoding on CPU
generated_text = gpt_tokenizer.decode(generated_ids, skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: There was a time when I saw a cow in the office, and I thought, 'Oh, that's a cow!' And then I saw a cow in the office, and I thought, 'Oh, that's a cow!' And then I


# Save a model